# &#x1F4D1; &nbsp; <span style="color:red"> Reflections. Deep Learning. Lesson 2</span>

##   &#x1F916; &nbsp; <span style="color:red">Links</span>

Logistic Regression: http://ufldl.stanford.edu/tutorial/supervised/LogisticRegression/

Improving the way neural networks learn: http://neuralnetworksanddeeplearning.com/chap3.html

tensorflow.examples https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples

Deep Learning. An MIT Press book: http://www.deeplearningbook.org/

Deep Machine Learning—A New Frontier in Artificial Intelligence Research: http://web.eecs.utk.edu/~itamar/Papers/DML_Arel_2010.pdf

Code samples for "Neural Networks and Deep Learning" https://github.com/mnielsen/neural-networks-and-deep-learning

##   &#x1F916; &nbsp; <span style="color:red">Libraries</span>

In [1]:
import numpy as np
import scipy as sp

In [2]:
from __future__ import print_function
import matplotlib.pyplot as plt

import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


##   &#x1F916; &nbsp; <span style="color:red">Deep Neural Networks</span>

In [41]:
# Quiz: Number Of Parameters
28*28*10 + 10

7850

In [42]:
# Quiz: Rectified Linear Units
# N2

Note: Depicted above is a "2-layer" neural network:

The first layer effectively consists of the set of weights and biases applied to X and passed through ReLUs. The output of this layer is fed to the next one, but is not observable outside the network, hence it is known as a hidden layer.
The second layer consists of the weights and biases applied to these intermediate outputs, followed by the softmax function to generate probabilities.

##   &#x1F916; &nbsp; <span style="color:red">Assignment 2: Stochastic Gradient Descent</span>